In [1]:
import pandas as pd
from flot_util import *
from flot_visualization import *
from flot_prediction import *
from flot_feature_engineering import *

In [2]:
df = pd.read_csv(r'C:\Users\rcpsi\OneDrive\Documents\langchain\Flotation\dados\Flotacao_Dados_Final.csv',sep=';')

## Data cleaning

In [ ]:
df = parse_time(df)
df = remove_missing(df)
df = filter_by_ph(df)
df = filter_by_flow(df)
df_clean = drop_consecutive_duplicates_tolerance(df)

#df_clean = add_time_features(df_clean)
#df_clean = add_ph_features(df_clean)
#df_clean = add_dosage_flow_features(df_clean)
#df_clean = add_cell_level_stats(df_clean)
#df_clean = add_rolling_features(df_clean)
df_clean = add_lag_silica_features(df_clean)

#df_clean = add_delta_silica_regular(df_clean)

In [4]:
df_clean.shape

(2618, 39)

In [5]:
df_clean.head()

,inicio,fim,conc_silica,conc_fe,param_dosagem_amido,dosagem_amina_conc_magnetica,ph_flotacao_linha01,ph_flotacao_linha02,densidade_alimentacao_flotacao,nivel_celula_li640101,...,cell_level_min,cell_level_range,dosagem_amina_conc_magnetica_rollmean_3,vazao_alimentacao_flotacao_rollmean_3,densidade_alimentacao_flotacao_rollmean_3,param_dosagem_amido_rollmean_3,dosagem_amina_conc_magnetica_rollmean_5,vazao_alimentacao_flotacao_rollmean_5,densidade_alimentacao_flotacao_rollmean_5,param_dosagem_amido_rollmean_5
0,2024-02-27 00:00:00,2024-02-27 00:20:00,2.72,64.59,492.50,152.54,9.47,7.99,1.60,369.44,...,328.20,41.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-02-27 01:00:00,2024-02-27 01:20:00,1.23,67.40,492.50,147.58,9.60,7.95,1.57,369.84,...,332.58,37.26,152.540000,907.140000,1.600,492.500000,152.5400,907.140000,1.6000,492.500000
2,2024-02-27 03:00:00,2024-02-27 03:20:00,1.93,65.09,491.67,135.00,9.50,7.92,1.60,368.38,...,340.50,27.88,150.060000,792.560000,1.585,492.500000,150.0600,792.560000,1.5850,492.500000
3,2024-02-27 05:00:00,2024-02-27 05:20:00,2.04,65.46,491.67,146.53,9.38,9.50,1.60,369.72,...,359.22,10.50,145.040000,811.103333,1.590,492.223333,145.0400,811.103333,1.5900,492.223333
4,2024-02-27 07:00:00,2024-02-27 07:20:00,4.56,63.75,491.67,150.75,9.43,9.52,1.60,370.38,...,333.77,36.61,143.036667,800.046667,1.590,491.946667,145.4125,826.820000,1.5925,492.085000


## Exploratory data analysis

In [6]:
plot_time_series_with_gaps(df_clean,'inicio','conc_silica')

In [7]:
# Usage examples:
# plot_time_series(df_clean, 'inicio', 'conc_silica')
plot_histogram(df_clean, 'conc_silica')
# plot_scatter(df_clean, 'vazao_alimentacao_flotacao', 'conc_silica')
# plot_pairwise(df_clean, ['conc_silica', 'conc_fe', 'dosagem_amina_conc_magnetica', 'vazao_alimentacao_flotacao'])

In [8]:
# df_clean.columns

In [9]:
plot_scatter(df_clean, 'ph_flotacao_linha01', 'conc_silica')

In [10]:
plot_scatter(df_clean, 'vazao_alimentacao_flotacao', 'conc_silica')

## Prediction

In [11]:
df_clean.columns

Index(['inicio', 'fim', 'conc_silica', 'conc_fe', 'param_dosagem_amido',
       'dosagem_amina_conc_magnetica', 'ph_flotacao_linha01',
       'ph_flotacao_linha02', 'densidade_alimentacao_flotacao',
       'nivel_celula_li640101', 'TO_LI6401_02', 'nivel_celula_li640201',
       'nivel_celula_li640202', 'nivel_celula_li641101',
       'nivel_celula_li641102', 'nivel_celula_li641201',
       'nivel_celula_li641202', 'nivel_celula_li642101',
       'nivel_celula_li642201', 'nivel_celula_li643101',
       'nivel_celula_li643201', 'vazao_alimentacao_flotacao', 'operacao',
       'duration_min', 'hour', 'dayofweek', 'ph_diff', 'dosage_flow_ratio',
       'cell_level_max', 'cell_level_min', 'cell_level_range',
       'dosagem_amina_conc_magnetica_rollmean_3',
       'vazao_alimentacao_flotacao_rollmean_3',
       'densidade_alimentacao_flotacao_rollmean_3',
       'param_dosagem_amido_rollmean_3',
       'dosagem_amina_conc_magnetica_rollmean_5',
       'vazao_alimentacao_flotacao_rollmean_5'

In [12]:
features = df_clean.columns.tolist()
drops = ['inicio','fim','conc_fe', 'operacao','duration_min']
target = ['conc_silica']    

features = list(set(features) - set(drops))
features = list(set(features) - set(target))

print(df_clean.shape)
df_clean.dropna(inplace=True)
print(df_clean.shape)
df_clean.drop(columns=drops, inplace=True)


(2618, 39)
(2617, 39)


In [13]:
res = evaluate_models(df_clean, features, target=target[0])
res

,model,MAE,RMSE,R2,MAPE (%)
0,LinearRegression,1.655626,2.955642,0.028960,72.339594
1,RandomForest,1.541577,2.855301,0.093772,65.456255
2,XGBoost,1.665358,2.958616,0.027004,69.731349


In [14]:
#res_weights = evaluate_models_weights(df_clean, features, target=target[0])
#res_weights

In [15]:
res_q = evaluate_models_quantile(df_clean, features, target=target[0], show_residuals=False, quantile=0.9)
res_q

,model,MAE,RMSE,R2,MAPE (%)
0,LinearRegression,1.724101,3.010870,0.095229,70.949709
1,RandomForest,1.690328,2.950801,0.130970,68.650828
2,XGBoost,1.810544,3.097897,0.042170,73.003105
3,QuantileGBR_0.9,2.527923,3.361046,-0.127466,138.971618


In [16]:
res_q = evaluate_models_quantile(df_clean, features, target=target[0], show_residuals=False, quantile=0.95)
res_q

,model,MAE,RMSE,R2,MAPE (%)
0,LinearRegression,1.590549,3.035816,0.092076,67.485387
1,RandomForest,1.524455,2.939124,0.148990,63.656254
2,XGBoost,1.599820,2.997565,0.114811,65.750292
3,QuantileGBR_0.95,4.095657,4.748331,-1.221164,245.959658


In [17]:
res_q = evaluate_models_quantile(df_clean, features, target=target[0], show_residuals=False, quantile=0.5)
res_q

,model,MAE,RMSE,R2,MAPE (%)
0,LinearRegression,1.564611,2.838495,0.095917,69.839045
1,RandomForest,1.493517,2.748830,0.152133,64.919370
2,XGBoost,1.681440,2.937519,0.031737,73.442042
3,QuantileGBR_0.5,1.339305,2.893519,0.060526,41.552974


In [18]:
res_tweedie = evaluate_tweedie_models(df_clean, features, target=target[0], power=1.5,show_residuals=True)
res_tweedie

c:\Users\rcpsi\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\genmod\families\links.py:13: FutureWarning:

The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.



c:\Users\rcpsi\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning:

Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.

  File "c:\Users\rcpsi\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "c:\Users\rcpsi\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\rcpsi\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 1036, in __init__
 

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000721 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7231
[LightGBM] [Info] Number of data points in the train set: 2093, number of used features: 33
[LightGBM] [Info] Start training from score 0.995666


,model,MAE,RMSE,R2,MAPE (%)
0,GLM_Gamma_Log,1.506133,2.513967,0.110746,73.676005
1,Sklearn_Tweedie(p=1.5),1.524683,2.518593,0.107470,75.668650
2,XGB_Tweedie(p=1.5),1.309830,2.452795,0.153496,50.199067
3,LGBM_Tweedie(p=1.5),1.303841,2.407238,0.184649,49.702308
